In [ ]:
import os
import glob

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
from scipy.ndimage import map_coordinates

%matplotlib inline

In [ ]:
def get_lmsia_data(year):
    idir = f'/data2/Anton/sia/cdr_1991_2023/nc/{year}'
    ifile = glob.glob(f'{idir}/*1231.nc')[0]
    with Dataset(ifile) as ds:
        sia = ds.variables['sia'][0]
        x = ds.variables['x'][:]
        y = ds.variables['y'][:]
        status_flag = ds.variables['status_flag'][0]
        conc = []
        for i in range(1, 8):
            if f'conc_{i}yi' in ds.variables:
                conc.append(ds.variables[f'conc_{i}yi'][0])
    conc = np.array(conc)[::-1]
    land = np.where(status_flag == 1, 1, np.nan)
    return sia, x, y, land, conc

def get_nsidc_data(year):
    ifile = f'/Data/sat/downloads/NSIDC_iceage/iceage_nh_12.5km_{year}0101_{year}1231_v4.1.nc'
    if os.path.exists(ifile):
        with Dataset(ifile) as ds:
            age_of_sea_ice = ds.variables['age_of_sea_ice'][-1]
            nsidc_x = ds.variables['x'][:]
            nsidc_y = ds.variables['y'][:]

        nsidc_age = np.where(age_of_sea_ice == 0, np.nan, age_of_sea_ice)
        nsidc_land = np.where(age_of_sea_ice >= 20, 1, np.nan)
        nsidc_age[nsidc_age >= 20] = np.nan
        return nsidc_age, nsidc_x, nsidc_y, nsidc_land
    else:
        return None, None, None, None

In [ ]:
min_conc = 0.15
years = [1994, 1999, 2004, 2009, 2014, 2019, 2024]

fig, axs = plt.subplots(3,len(years), figsize=(3 * len(years), 11), sharex=True, sharey=True)

for iyear, year in enumerate(years):
    sia, x, y, land, conc = get_lmsia_data(year)

    tot_conc = np.sum(conc, axis=0)
    max_age = len(conc) - np.argmax(conc > min_conc, axis=0).astype(float)
    max_age[(tot_conc < min_conc) + np.isnan(tot_conc)] = np.nan

    axs[0, iyear].imshow(sia, cmap='viridis', clim=[0, 5], extent=[x[0], x[-1], y[-1], y[0]], interpolation='none')
    axs[0, iyear].imshow(land, cmap='gray', clim=[0, 2], extent=[x[0], x[-1], y[-1], y[0]], interpolation='none')

    axs[1, iyear].imshow(max_age, cmap='viridis', clim=[1, 7], extent=[x[0], x[-1], y[-1], y[0]], interpolation='none')
    axs[1, iyear].imshow(land, cmap='gray', clim=[0, 2], extent=[x[0], x[-1], y[-1], y[0]], interpolation='none')

    nsidc_age, nsidc_x, nsidc_y, nsidc_land = get_nsidc_data(year)
    if nsidc_age is not None:
        axs[2, iyear].imshow(nsidc_age, extent=[nsidc_x[0], nsidc_x[-1], nsidc_y[-1], nsidc_y[0]], cmap='viridis', clim=[1, 7], interpolation='none')
        axs[2, iyear].imshow(nsidc_land, cmap='gray', extent=[nsidc_x[0], nsidc_x[-1], nsidc_y[-1], nsidc_y[0]], clim=[0,2], interpolation='none')

    axs[0, iyear].set_title(year+1)

for ax in axs.flat:
    ax.set_xlim([-1.9e6, 2.1e6])
    ax.set_ylim([-2.35e6, 2.5e6])
    ax.set_axis_off()

#axs[0, 0].set_ylabel('LM SIA, mean age')
#axs[1, 0].set_ylabel('LM SIA, max age')
#axs[2, 0].set_ylabel('NSIDC, sea ice age')
plt.tight_layout()
plt.show()

In [ ]:
year = 2020
sia, x, y, land, conc = get_lmsia_data(year)
nsidc_age, nsidc_x, nsidc_y, nsidc_land = get_nsidc_data(year)
nsidc_pixel_area = np.mean(np.diff(nsidc_x) * np.diff(nsidc_y)) / 1e6 # km2

rows, cols = np.mgrid[0:sia.shape[0], 0:sia.shape[1]]
rgi_rows = RegularGridInterpolator((y, x), rows, method='nearest', bounds_error=False, fill_value=None)
rgi_cols = RegularGridInterpolator((y, x), cols, method='nearest', bounds_error=False, fill_value=None)
rows_nsidc = rgi_rows((np.meshgrid(nsidc_y, nsidc_x, indexing='ij')))
cols_nsidc = rgi_cols((np.meshgrid(nsidc_y, nsidc_x, indexing='ij')))

land_pro = map_coordinates(np.where(land == 1, 1, 0), (rows_nsidc, cols_nsidc), order=0, mode='nearest')
conc_mask_pro = map_coordinates(np.where(np.isfinite(conc[0]), 1, 0), (rows_nsidc, cols_nsidc), order=0, mode='nearest')
nsidc_land_int = np.where(nsidc_land == 1, 1, 0)
joint_flag = nsidc_land_int + 2 * land_pro + 4 * conc_mask_pro
# water or sea ice processed on both NSIDC and LMSIA datasets at NSIDC resolution
valid_mask = joint_flag == 4

In [ ]:
sia_areas_ts = []
nsidc_area_ts = []
years = range(1996, 2024)
for year in years:
    sia, x, y, land, conc = get_lmsia_data(year)
    nsidc_age, nsidc_x, nsidc_y, nsidc_land = get_nsidc_data(year)
    conc_pro = np.array([
        map_coordinates(c, (rows_nsidc, cols_nsidc), order=0, mode='nearest')
        for c in conc
    ])
    conc_pro[:, ~valid_mask] = np.nan
    sia_areas = np.nansum(conc_pro, axis=(1, 2))

    nsidc_age[~valid_mask] = np.nan
    nsidc_age_areas = np.diff([(nsidc_age >= i).sum() for i in range(1, 9)][::-1])
    sia_areas_ts.append(sia_areas)
    nsidc_area_ts.append(nsidc_age_areas)
sia_areas_ts = np.array(sia_areas_ts).T * nsidc_pixel_area
nsidc_area_ts = np.array(nsidc_area_ts).T * nsidc_pixel_area

In [ ]:
# Plot the time series as a stacked bar plot
fig, ax = plt.subplots(figsize=(8, 3))

# Create labels for age classes
age_labels = ['FYI', '1', ' 2', '3', '4', '5', '>=6'][::-1]

# Create x-axis for years
x = np.array(list(years))+1

# Create a stacked bar plot
bottom = np.zeros(len(years))
bars = []
colors = plt.cm.viridis(np.linspace(0, 1, sia_areas_ts.shape[0]))
for i, age_data in enumerate(sia_areas_ts):
    bars.append(ax.bar(x-0.2, age_data, bottom=bottom, label=age_labels[i], color=colors[i], width=0.35))
    bottom += age_data

bottom = np.zeros(len(years))
bars = []
colors = plt.cm.viridis(np.linspace(0, 1, sia_areas_ts.shape[0]))
for i, age_data in enumerate(nsidc_area_ts):
    bars.append(ax.bar(x+0.2, age_data, bottom=bottom, color=colors[i], width=0.35))
    bottom += age_data


# Add labels and title
ax.set_ylabel('Sea Ice Area (km²)', fontsize=12)
ax.set_title('Sea Ice Area by Age Class (1996-2023)', fontsize=14)

# Add legend
ax.legend(loc='upper right')

# Add grid
ax.grid(True, linestyle='--', alpha=0.7)

ax.set_xlim([1996, 2030])
# Adjust x-axis ticks to show every 3 years
plt.xticks(x, rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
from cmocean import cm

In [ ]:
cm.ice

In [ ]:

plt.plot(sia_areas, '.-')
plt.plot(nsidc_age_areas, '.-')